# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables <font color=red>event_datafile_new.csv</font>

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))


6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
# Create a Keyspace
try: 
    session.execute("CREATE KEYSPACE IF NOT EXISTS udacity \
                    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}")
except Exception as e:
    print('Error at Create Keyspace')
    print(e)

#### Set Keyspace

In [6]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

--- 
## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during **sessionId = 338, and itemInSession  = 4**


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for **userid = 10, sessionid = 182**
    

3. Give me every user name (first and last) in my music app history who listened to the **song 'All Hands Against His Own'**
---

## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create table names `song_by_sessions` beacuse this query want to get song information by given `session_id` and `item_in_session`
Use `session_id` as partition key and `item_is_session` as clustering key.

**Columns names is** (session_id , item_in_session , artist , song , length)

In [7]:
# Create denomrlized table
try:
    # drop if exists
    row = session.execute('DROP TABLE IF EXISTS song_by_sessions')
    row = session.execute('CREATE TABLE IF NOT EXISTS \
                        song_by_sessions (session_id int, item_in_session int, artist text, song text, length double, \
                          PRIMARY KEY (session_id, item_in_session))')
except Exception as e:
    print('Error at Create Table')
    print(e)        

#### Insert data from <font color=red>event_datafile_new.csv</font> into table `song_by_sessions`

In [8]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # make inserting query
        query = "INSERT INTO song_by_sessions (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        values = (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
        # execute query
        session.execute(query, values)
print('Finished Inserting')
 

Finished Inserting


#### Test query by `session_id=338` and `item_in_session=4`
Expected output: 
```
Row(session_id=338, item_in_session=4, artist='Faithless', length=495.3073, song='Music Matters (Mark Knight Dub)')
```

In [9]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    query_result = session.execute('SELECT * FROM song_by_sessions WHERE session_id=338 and item_in_session=4')

except Exception as e:
    print('Error at SELECT')
    print(e)

for row in query_result:
    print(row)

Row(session_id=338, item_in_session=4, artist='Faithless', length=495.3073, song='Music Matters (Mark Knight Dub)')


## 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create table names `song_by_user_id_session_id` beacuse this query want to get song information by given `user_id` and `session_id`
Use composite partition key of (`user_id`, `session_id`) and use `item_in_session` to make it unique and able to clustering if needed.

**Columns names is** (user_id, session_id, item_in_session, artist, song, first_name, last_name)

In [10]:
# Create denomrlized table
try:
    # drop if exists
    row = session.execute('DROP TABLE IF EXISTS song_by_user_id_session_id')
    row = session.execute('CREATE TABLE IF NOT EXISTS \
                          song_by_user_id_session_id (user_id int, session_id int, item_in_session int, artist text, song text, \
                          first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))')
except Exception as e:
    print('Error at Create Table')
    print(e)
         

#### Insert data from <font color=red>event_datafile_new.csv</font> into table `song_by_user_id_session_id`

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # make inserting query
        query = "INSERT INTO song_by_user_id_session_id (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
        query = query + "VALUES (%s, %s,%s, %s, %s, %s, %s)"
        values = (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])
        # execute query
        session.execute(query, values)
print('Finished Inserting')

Finished Inserting


#### Test query by `user_id=10` and `session_id=182`
Expected output: 
```
Row(user_id=10, session_id=182, item_in_session=0, artist='Down To The Bone', first_name='Sylvie', last_name='Cruz', song="Keep On Keepin' On")
Row(user_id=10, session_id=182, item_in_session=1, artist='Three Drives', first_name='Sylvie', last_name='Cruz', song='Greece 2000')
Row(user_id=10, session_id=182, item_in_session=2, artist='Sebastien Tellier', first_name='Sylvie', last_name='Cruz', song='Kilometer')
Row(user_id=10, session_id=182, item_in_session=3, artist='Lonnie Gordon', first_name='Sylvie', last_name='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')
```

In [12]:
try:
    query_result = session.execute('SELECT * FROM song_by_user_id_session_id WHERE user_id=10 and session_id=182')

except Exception as e:
    print('Error at SELECT')
    print(e)

for row in query_result:
    print(row)

Row(user_id=10, session_id=182, item_in_session=0, artist='Down To The Bone', first_name='Sylvie', last_name='Cruz', song="Keep On Keepin' On")
Row(user_id=10, session_id=182, item_in_session=1, artist='Three Drives', first_name='Sylvie', last_name='Cruz', song='Greece 2000')
Row(user_id=10, session_id=182, item_in_session=2, artist='Sebastien Tellier', first_name='Sylvie', last_name='Cruz', song='Kilometer')
Row(user_id=10, session_id=182, item_in_session=3, artist='Lonnie Gordon', first_name='Sylvie', last_name='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


## 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create table names `listened_user_by_song` beacuse this query want to get listened information by given `song` name
#### We need to add column `user_id` and use it as clustering key to make the row unique. We can't use `first_name` and `last_name` because it can be duplicated name.

**Columns names is** (user_id, session_id, item_in_session, artist, song, first_name, last_name)

In [13]:
# Create denomrlized table
try:
    # drop if exists
    row = session.execute('DROP TABLE IF EXISTS listened_user_by_song')
    row = session.execute('CREATE TABLE IF NOT EXISTS \
                          listened_user_by_song (song text, user_id int, first_name text, last_name text, \
                          PRIMARY KEY (song, user_id))')
except Exception as e:
    print('Error at Create Table')
    print(e)
                    

#### Insert data from <font color=red>event_datafile_new.csv</font> into table `song_by_user_id_session_id`

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # make inserting query
        query = "INSERT INTO listened_user_by_song (song, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        values = (line[9], int(line[10]), line[1], line[4])
        # execute query
        session.execute(query, values)
print('Finished Inserting')

#### Test query by `song='All Hands Against His Own'`
Expected output: 
```
Row(song='All Hands Against His Own', user_id=29, first_name='Jacqueline', last_name='Lynch')
Row(song='All Hands Against His Own', user_id=80, first_name='Tegan', last_name='Levine')
Row(song='All Hands Against His Own', user_id=95, first_name='Sara', last_name='Johnson')
```

In [ ]:
try:
    query_result = session.execute("SELECT * FROM listened_user_by_song WHERE song='All Hands Against His Own'")

except Exception as e:
    print('Error at SELECT')
    print(e)

for row in query_result:
    print(row)

---
#### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute('DROP TABLE IF EXISTS song_by_sessions')
except Exception as e:
    print(e)
try:
    session.execute('DROP TABLE IF EXISTS song_by_user_id_session_id')
except Exception as e:
    print(e)
try:
    session.execute('DROP TABLE IF EXISTS listened_user_by_song')
except Exception as e:
    print(e)


#### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()